In [1]:
import pandas as pd
import numpy as np
import json
import math

import base64
from io import BytesIO
from PIL import Image

from jinja2 import Environment, Template, FileSystemLoader

In [8]:
env = Environment()
env.filters['jsonify'] = json.dumps

In [2]:
df_numeric = pd.read_pickle('data/dataframes/pokemon_numeric.pickle')
df_names = pd.read_pickle('data/dataframes/pokemon_names.pickle')
df_sprites = pd.read_pickle('data/dataframes/pokemon_sprites.pickle.bz2', compression='bz2')
df_reduced = pd.read_pickle('data/dataframes/pokemon_pca_tsne_umap.pickle')

In [4]:
df_data = df_numeric.merge(df_names, left_index=True, right_index=True).merge(df_sprites, left_on='pokedex_number', right_on='pokedex_number').merge(df_reduced, left_index=True, right_index=True)

In [13]:
df_data.head()

,attack,base_egg_steps,base_happiness,base_total,capture_rate,defense,experience_growth,height_m,hp,sp_attack,...,is_legendary,name,pokedex_number,image_data,pca-one,pca-two,tsne-one,tsne-two,umap-one,umap-two
0,49,5120,70,318,45,49,1059860,0.7,45,65,...,0,Bulbasaur,1,"[[[255, 255, 255, 0], [255, 255, 255, 0], [255...",-0.834394,-0.507250,-2.762704,10.562872,7.062129,-1.793816
1,62,5120,70,405,45,63,1059860,1.0,60,80,...,0,Ivysaur,2,"[[[255, 255, 255, 0], [255, 255, 255, 0], [255...",-0.744409,-0.571682,-2.760835,10.539059,6.989386,-1.767467
2,100,5120,70,625,45,123,1059860,2.0,80,122,...,0,Venusaur,3,"[[[255, 255, 255, 0], [255, 255, 255, 0], [255...",-0.497195,-0.735575,-2.780355,10.516722,7.006114,-1.809384
3,52,5120,70,309,45,43,1059860,0.6,39,60,...,0,Charmander,4,"[[[255, 255, 255, 0], [255, 255, 255, 0], [255...",0.005951,0.136791,-15.378108,3.138825,0.927519,-5.502607
4,64,5120,70,405,45,58,1059860,1.1,58,80,...,0,Charmeleon,5,"[[[255, 255, 255, 0], [255, 255, 255, 0], [255...",0.112033,0.057822,-15.471931,3.271016,0.903426,-5.415740


In [5]:
limits = {}
limits['xmin'], limits['xmax'] = (math.floor(df_data['tsne-one'].min()), math.ceil(df_data['tsne-one'].max()))
limits['ymin'], limits['ymax'] = (math.floor(df_data['tsne-one'].min()), math.ceil(df_data['tsne-one'].max()))

In [14]:
colors = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f']
generations = []
for gen in df_data['generation'].unique():
    color = colors[gen - 1]
    name = 'Generation {}'.format(gen)
    series = []
    for index, row in df_data[df_data['generation'] == gen].iterrows():
        pil_img = Image.fromarray(row['image_data'])
        pil_img.thumbnail(size=(50,50))
        buff = BytesIO()
        pil_img.save(buff, format="PNG")
        image_string = base64.b64encode(buff.getvalue()).decode("utf-8")
        series.append({
            'x': row['tsne-one'],
            'y': row['tsne-two'],
            'name': row['name'],
            'attack': row['attack'],
            'defense': row['defense'],
            'hp': row['hp'],
            'image_data': image_string
        })
    generations.append({
        'name': name,
        'color': color,
        'data': series
    })

In [15]:
with open('static/highcharts_scatter_2d_template.html') as file_:
    template = file_.read()
    with open('static/highcharts_scatter_2d_gen.html', 'w') as outfile_:
        outfile_.write(env.from_string(template).render(series=generations, limits=limits))

In [19]:
df_types = pd.read_pickle('data/dataframes/pokemon_types.pickle')
df_types.columns

Index(['bug', 'dark', 'dragon', 'electric', 'fairy', 'fighting', 'fire',
       'flying', 'ghost', 'grass', 'ground', 'ice', 'normal', 'poison',
       'psychic', 'rock', 'steel', 'water'],
      dtype='object')